[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/oreilly-japan/RecommenderSystems/blob/main/chapter5/colab/Popularity.ipynb)

# 인기도 순 추천
## 인기도의 정의
* 여기에서는 평갓값이 높은 것을 인기가 높은 영화로 간주합니다
* 인기도는 '클릭 수가 많은 것', '구매가 많은 것', '평갓값이 높은 것' 등 다양하게 정의할 수 있으므로 자사 서비스에 맞춰 적합한 정의를 사용합니다.


In [1]:
# Colab용 notebook입니다. 이 notebook 한 장에서 여러 데이터의 다운로드부터, 추천까지 완결하도록 되어 있습니다(예측 평가는 미포함)
# MovieLens 데이터를 아직 다운로드 하지 않았다면, 이 셀을 실행해서 다운로드합니다.
# MovieLens 데이터 분석은 data_download.ipynb를 참조합니다.

# 데이터 다운로드와 압축 풀기
!wget -nc --no-check-certificate https://files.grouplens.org/datasets/movielens/ml-10m.zip -P ../data
!unzip -n ../data/ml-10m.zip -d ../data/

--2022-12-27 05:40:34--  https://files.grouplens.org/datasets/movielens/ml-10m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 65566137 (63M) [application/zip]
Saving to: ‘../data/ml-10m.zip’

ml-10m.zip          100%[===================>]  62.53M  92.1MB/s    in 0.7s    

2022-12-27 05:40:35 (92.1 MB/s) - ‘../data/ml-10m.zip’ saved [65566137/65566137]

Archive:  ../data/ml-10m.zip
   creating: ../data/ml-10M100K/
  inflating: ../data/ml-10M100K/allbut.pl  
  inflating: ../data/ml-10M100K/movies.dat  
  inflating: ../data/ml-10M100K/ratings.dat  
  inflating: ../data/ml-10M100K/README.html  
  inflating: ../data/ml-10M100K/split_ratings.sh  
  inflating: ../data/ml-10M100K/tags.dat  


In [2]:
# Movielens 데이터 로딩(데이터량이 많으므로, 로딩에 시간이 걸릴 수 있습니다)
import pandas as pd

# movieID와 제목만 사용
m_cols = ['movie_id', 'title', 'genre']
movies = pd.read_csv('../data/ml-10M100K/movies.dat', names=m_cols, sep='::' , encoding='latin-1', engine='python')

# genre를 list 형식으로 저장한다
movies['genre'] = movies.genre.apply(lambda x:x.split('|'))


# 사용자가 부여한 영화의 태그 정보를 로딩한다
t_cols = ['user_id', 'movie_id', 'tag', 'timestamp']
user_tagged_movies = pd.read_csv('../data/ml-10M100K/tags.dat', names=t_cols, sep='::', engine='python')

# tag를 소문자로 바꾼다
user_tagged_movies['tag'] = user_tagged_movies['tag'].str.lower()


# tag를 영화별로 list 형식으로 저장한다
movie_tags = user_tagged_movies.groupby('movie_id').agg({'tag':list})

# 태그 정보를 결합한다
movies = movies.merge(movie_tags, on='movie_id', how='left')

# 평갓값 데이터만 로딩한다
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('../data/ml-10M100K/ratings.dat', names=r_cols, sep='::', engine='python')


# 데이터량이 많으므로 사용자수를 1000으로 줄여서 시험해본다
valid_user_ids = sorted(ratings.user_id.unique())[:1000]
ratings = ratings[ratings["user_id"].isin(valid_user_ids)]


# 영화 데이터와 평가 데이터를 결합한다
movielens = ratings.merge(movies, on='movie_id')

print(f'unique_users={len(movielens.user_id.unique())}, unique_movies={len(movielens.movie_id.unique())}')

# 학습용과 데이터용으로 데이터를 나눈다
# 각 사용자의 최근 5건의 영화를 평가용으로 사용하고, 나머지는 학습용으로 사용한다
# 우선, 각 사용자가 평가한 영화의 순서를 계산한다
# 최근 부여한 영화부터 순서를 부여한다(1에서 시작)

movielens['timestamp_rank'] = movielens.groupby(
    'user_id')['timestamp'].rank(ascending=False, method='first')
movielens_train = movielens[movielens['timestamp_rank'] > 5]
movielens_test = movielens[movielens['timestamp_rank']<= 5]

unique_users=1000, unique_movies=6736


In [3]:
# 평가 수의 임곗값
minimum_num_rating = 200


In [4]:
import numpy as np

# 평갓값의 평균이 높은 영화를 확인한다
# 평가 수가 1건인 영화가 상위에 여럿 나타난다
movie_stats = movielens_train.groupby(['movie_id', 'title']).agg({'rating': [np.size, np.mean]})
movie_stats.sort_values(by=('rating', 'mean'), ascending=False).head()

rating     
                                                              size mean
movie_id title                                                         
4095     Cry Freedom (1987)                                      1  5.0
7227     Trouble with Angels, The (1966)                         1  5.0
27255    Wind Will Carry Us, The (Bad ma ra khahad bord)...      1  5.0
4453     Michael Jordan to the Max (2000)                        2  5.0
3415     Mirror, The (Zerkalo) (1975)                            1  5.0

In [5]:
# 임곗값을 도입해 평가 수가 적은 영화를 제거한다
# 쇼생크 탈출이나 7인의 사무라이 등 익숙한 영화가 상위에 나타난다
movie_stats = movielens_train.groupby(['movie_id', 'title']).agg({'rating': [np.size, np.mean]})
atleast_flg = movie_stats['rating']['size'] >= 100
movies_sorted_by_rating = movie_stats[atleast_flg].sort_values(by=('rating', 'mean'), ascending=False)
movies_sorted_by_rating.head()

rating          
                                                       size      mean
movie_id title                                                       
318      Shawshank Redemption, The (1994)               424  4.491745
50       Usual Suspects, The (1995)                     334  4.459581
912      Casablanca (1942)                              163  4.444785
904      Rear Window (1954)                             129  4.441860
2019     Seven Samurai (Shichinin no samurai) (1954)    104  4.408654

In [6]:
import numpy as np 
# 각 아이템별로 평균 평갓값을 계싼하고, 해당 평균 평갓값을 예측값으로 사용한다
movie_rating_average = movielens_train.groupby("movie_id").agg({"rating": np.mean})
# 테스트 데이터에 예측값을 저장한다. 테스트 데이터에만 존재하는 아이템의 예측 평갓값은 0으로 한다.
movie_rating_predict = movielens_test.merge(
    movie_rating_average, on="movie_id", how="left", suffixes=("_test", "_pred")
).fillna(0)

In [7]:
from collections import defaultdict

# 각 사용자에 대한 추천 영화는 해당 사용자가 아직 평가하지 않은 영화 중에서 평갓값이 높은 10작품으로 한다
# 단, 평가 건수가 작으면 노이즈가 크므로 minimum_num_rating건 이상 평가가 있는 영화로 필터링한다
pred_user2items = defaultdict(list)
user_watched_movies = movielens_train.groupby("user_id").agg({"movie_id": list})["movie_id"].to_dict()
movie_stats = movielens_train.groupby("movie_id").agg({"rating": [np.size, np.mean]})
atleast_flg = movie_stats["rating"]["size"] >= minimum_num_rating
movies_sorted_by_rating = (
    movie_stats[atleast_flg].sort_values(by=("rating", "mean"), ascending=False).index.tolist()
)

for user_id in movielens_train.user_id.unique():
    for movie_id in movies_sorted_by_rating:
        if movie_id not in user_watched_movies[user_id]:
            pred_user2items[user_id].append(movie_id)
        if len(pred_user2items[user_id]) == 10:
            break
pred_user2items

defaultdict(list,
            {139: [50, 858, 1193, 1214, 1200, 1291, 457, 150, 2396, 34],
             149: [318, 527, 260, 1193, 593, 2571, 1136, 2028, 1197, 2762],
             182: [260, 541, 1198, 1214, 1200, 47, 1036, 1291, 1210, 1240],
             215: [527, 590, 1073, 539, 253, 2683, 231],
             281: [858, 260, 1193, 2571, 2959, 1196, 2858, 1198, 1617, 1136],
             326: [318, 858, 527, 260, 1193, 2571, 541, 2959, 1196, 2858],
             351: [541, 2959, 2028, 2762, 1200, 3578, 47, 1291, 1240, 589],
             357: [318, 50, 858, 260, 1193, 593, 541, 2959, 1196, 2858],
             426: [1073, 141, 329],
             456: [858, 527, 260, 1193, 2571, 541, 2959, 1196, 2858, 1198],
             459: [318, 50, 858, 1193, 1198, 1617, 1197, 608, 457, 1265],
             494: [858, 260, 1193, 2571, 541, 2959, 1196, 2858, 1198, 1617],
             517: [858, 527, 260, 1193, 593, 2571, 541, 2959, 1196, 2858],
             524: [527, 260, 1193, 593, 1196, 1198, 1617, 11

In [8]:
# user_id=2인 사용자가 학습 데이터에 평가를 부여한 영화 목록
movielens_train[movielens_train.user_id==2]

user_id  movie_id  rating  timestamp  \
4732        2       110     5.0  868245777   
5246        2       260     5.0  868244562   
5798        2       590     5.0  868245608   
6150        2       648     2.0  868244699   
6531        2       733     3.0  868244562   
6813        2       736     3.0  868244698   
7113        2       780     3.0  868244698   
7506        2       786     3.0  868244562   
7661        2       802     2.0  868244603   
7779        2       858     2.0  868245645   
8077        2      1049     3.0  868245920   
8127        2      1073     3.0  868244562   
8381        2      1210     4.0  868245644   
8771        2      1356     3.0  868244603   
9097        2      1544     3.0  868245920   

                                                  title  \
4732                                  Braveheart (1995)   
5246  Star Wars: Episode IV - A New Hope (a.k.a. Sta...   
5798                          Dances with Wolves (1990)   
6150                         Mission: Impossible (1996)   
6531                                   Rock, The (1996)   
6813                                     Twister (1996)   
7113               Independence Day (a.k.a. ID4) (1996)   
7506                                      Eraser (1996)   
7661                                  Phenomenon (1996)   
7779                              Godfather, The (1972)   
8077                 Ghost and the Darkness, The (1996)   
8127         Willy Wonka & the Chocolate Factory (1971)   
8381  Star Wars: Episode VI - Return of the Jedi (1983)   
8771                    Star Trek: First Contact (1996)   
9097  Lost World: Jurassic Park, The (Jurassic Park ...   

                                              genre  \
4732                           [Action, Drama, War]   
5246                    [Action, Adventure, Sci-Fi]   
5798                    [Adventure, Drama, Western]   
6150         [Action, Adventure, Mystery, Thriller]   
6531                  [Action, Adventure, Thriller]   
6813         [Action, Adventure, Romance, Thriller]   
7113               [Action, Adventure, Sci-Fi, War]   
7506                      [Action, Drama, Thriller]   
7661                               [Drama, Romance]   
7779                                 [Crime, Drama]   
8077                            [Action, Adventure]   
8127           [Children, Comedy, Fantasy, Musical]   
8381                    [Action, Adventure, Sci-Fi]   
8771          [Action, Adventure, Sci-Fi, Thriller]   
9097  [Action, Adventure, Horror, Sci-Fi, Thriller]   

                                                    tag  timestamp_rank  
4732  [bullshit history, medieval, bloodshed, hero, ...             8.0  
5246  [desert, quotable, lucas, gfei own it, seen mo...            17.0  
5798  [afi 100, lame, native, biopic, american india...            11.0  
6150  [confusing, confusing plot, memorable sequence...            12.0  
6531  [gfei own it, alcatraz, nicolas cage, sean con...            18.0  
6813  [disaster, disaster, storm, bill paxton, helen...            13.0  
7113  [action, alien invasion, aliens, will smith, a...            14.0  
7506  [arnold schwarzenegger, action, arnold, arnold...            19.0  
7661  [interesting concept, own, john travolta, john...            15.0  
7779  [oscar (best picture), marlon brando, classic,...             9.0  
8077  [lions, own, seen at the cinema, adventure, af...             6.0  
8127  [based on a book, roald dahl, based on a book,...            20.0  
8381  [desert, fantasy, sci-fi, space, lucas, gfei o...            10.0  
8771  [space, borg, futuristmovies.com, patrick stew...            16.0  
9097  [dinosaurs, dinosaurs, steven spielberg, borin...             7.0

In [9]:
pred_user2items[2]

[318, 50, 527, 1193, 593, 2571, 541, 2959, 1196, 2858]

In [10]:
# user_id=2에 대한 추천(318, 50, 527)
movies[movies.movie_id.isin([318, 50, 527])]

movie_id                             title                       genre  \
49         50        Usual Suspects, The (1995)  [Crime, Mystery, Thriller]   
315       318  Shawshank Redemption, The (1994)                     [Drama]   
523       527           Schindler's List (1993)                [Drama, War]   

                                                   tag  
49   [kevin spacey, ensemble cast, complicated, mus...  
315  [based on a short story, directorial debut, fr...  
523  [speilberg, drama, holocaust, steven spielberg...